In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Select only numeric columns from the market data (we don't want to scale non-numeric data)
market_data_numeric = df_market_data.select_dtypes(include=["float64", "int64"])

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler and transform the numeric data to standardize it (mean=0, std=1)
scaled_market_data = scaler.fit_transform(market_data_numeric)

In [6]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled_market_data)

# Copy the crypto names from the original DataFrame
scaled_df["coin_id"] = df_market_data.index

# Set the coin_id column as index
scaled_df = scaled_df.set_index("coin_id")

# Display the scaled DataFrame
scaled_df.head()


,0,1,2,3,4,5,6
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))  # We'll test k values from 1 to 11 for the elbow method


In [8]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    kmeans_model = KMeans(n_clusters=k, random_state=1)
    kmeans_model.fit(scaled_df)
    inertia.append(kmeans_model.inertia_)  # Inertia measures how tightly the data is clustered


In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)


In [10]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve to Find the Optimal k",
    xticks=k_values,
    width=700,
    height=400
)


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k is 4, as indicated by the “elbow point” on the inertia plot where the rate of decrease in inertia begins to level off. This suggests that 4 clusters provide the optimal balance between minimizing inertia and avoiding overfitting.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=1)  # Using k=4 based on elbow curve result


In [12]:
# Fit the K-Means model using the scaled DataFrame
kmeans.fit(scaled_df)  # Fit the model to the standardized cryptocurrency data


KMeans(n_clusters=4, random_state=1)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predicted_clusters = kmeans.predict(scaled_df)

# Print the resulting array of cluster values.
print(predicted_clusters)  # Each value represents the cluster assigned to a cryptocurrency


[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [14]:
# Create a copy of the scaled DataFrame
scaled_clustered_df = scaled_df.copy()  # We'll use this copy to add cluster labels


In [15]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaled_clustered_df["cluster"] = predicted_clusters  # Label each crypto with its assigned cluster

# Display the copy of the scaled DataFrame
scaled_clustered_df.head()


,0,1,2,3,4,5,6,cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [16]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scaled_clustered_df.hvplot.scatter(
    x="0",  # This column represents price_change_percentage_24h (after scaling)
    y="1",  # This column represents price_change_percentage_7d (after scaling)
    by="cluster",  # Color points by cluster assignment
    hover_cols=["coin_id"],  # Show coin ID on hover
    width=700,
    height=400,
    title="Crypto Clusters (K-Means, Scaled DataFrame)"
)


:NdOverlay   [cluster]
   :Scatter   [0]   (1,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)  # Reduce the scaled data to 3 principal components for optimal visualization and clustering


In [18]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
crypto_pca_data = pca.fit_transform(scaled_df)  # Perform dimensionality reduction

# View the scaled PCA data
crypto_pca_data[:5]  # Show the first five rows of the reduced 3D data


array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [19]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_  # This shows the proportion of variance explained by PC1, PC2, and PC3


array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is approximately 89.5%. This means that 89.5% of the information in the original dataset is preserved after reducing it to three principal components.

In [20]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(crypto_pca_data)

# Copy the crypto names from the original scaled DataFrame
pca_df["coin_id"] = scaled_df.index

# Set the coin_id column as index
pca_df = pca_df.set_index("coin_id")

# Display the scaled PCA DataFrame
pca_df.head()


,0,1,2
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [21]:
# Create a list with the number of k-values from 1 to 11
pca_k_values = list(range(1, 12))  # We'll evaluate cluster sizes from 1 to 11 using PCA-reduced data

In [22]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in pca_k_values:
    pca_kmeans_model = KMeans(n_clusters=k, random_state=1)
    pca_kmeans_model.fit(pca_df)
    pca_inertia.append(pca_kmeans_model.inertia_)  # Track clustering performance across k values


In [23]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    "k": pca_k_values,
    "inertia": pca_inertia
}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)  # Prepare for visualization


In [24]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve (PCA-Reduced Data)",
    xticks=pca_k_values,
    width=700,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k when using the PCA data is 4, as indicated by the elbow point where inertia significantly drops and begins to level off.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it does not differ. The best value for k using the original scaled data was also 4, confirming consistency across both approaches.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [25]:
# Initialize the K-Means model using the best value for k
pca_kmeans = KMeans(n_clusters=4, random_state=1)  # Using k=4 based on the elbow method for PCA-reduced data


In [26]:
# Fit the K-Means model using the PCA data
pca_kmeans.fit(pca_df)  # Train the model on the 3D PCA-reduced DataFrame


KMeans(n_clusters=4, random_state=1)

In [27]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_clusters = pca_kmeans.predict(pca_df)

# Print the resulting array of cluster values.
print(pca_clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [28]:
# Create a copy of the scaled PCA DataFrame
pca_clustered_df = pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pca_clustered_df["cluster"] = pca_clusters  # Assign predicted cluster labels

# Display the copy of the scaled PCA DataFrame
pca_clustered_df.head()


,0,1,2,cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [29]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
pca_clustered_df.hvplot.scatter(
    x="0",  # Principal Component 1 (PC1)
    y="1",  # Principal Component 2 (PC2)
    by="cluster",  # Color points by cluster label
    hover_cols=["coin_id"],  # Show coin_id on hover
    width=700,
    height=400,
    title="Crypto Clusters (K-Means, PCA-Reduced Data)"
)


:NdOverlay   [cluster]
   :Scatter   [0]   (1,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [30]:
# Composite plot to contrast the Elbow curves
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curves Comparison",
    label="Original Scaled Data"
) + pca_elbow_df.hvplot.line(
    x="k",
    y="inertia",
    label="PCA-Reduced Data"
)

:Layout
   .Curve.Original_Scaled_Data          :Curve   [k]   (inertia)
   .Curve.PCA_hyphen_minus_Reduced_Data :Curve   [k]   (inertia)

In [31]:
# Composite plot to contrast the clusters
scaled_clustered_df.hvplot.scatter(
    x="0",
    y="1",
    by="cluster",
    hover_cols=["coin_id"],
    width=700,
    height=400,
    title="Clusters (Original Scaled Data)"
) + pca_clustered_df.hvplot.scatter(
    x="0",
    y="1",
    by="cluster",
    hover_cols=["coin_id"],
    width=700,
    height=400,
    title="Clusters (PCA-Reduced Data)"
)

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [0]   (1,coin_id)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [0]   (1,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features through PCA helps simplify the dataset while retaining most of its important information. Visually, the clusters formed using PCA-reduced data are more compact and well-separated, indicating clearer distinctions between groups. This shows that dimensionality reduction can enhance clustering performance and visualization while reducing noise and computational complexity, without significantly compromising clustering accuracy.